In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from shutil import copyfile
from sklearn.model_selection import train_test_split
from IPython.display import SVG,display
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications import VGG16, DenseNet201, ResNet50, InceptionV3, InceptionResNetV2
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import load_img

/home/diegues/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def trainCNNModel(model, optimizer, factor, name):
    tensorboard = TensorBoard(log_dir='./Graph/' + name, histogram_freq=0, write_graph=True, write_images=True)
    best_checkpoint = ModelCheckpoint('./weights/' + name + '.h5', monitor='val_acc', save_best_only=True, verbose=1)
    earlystop = EarlyStopping(monitor='val_acc', patience=10, verbose=1)
    reducer = ReduceLROnPlateau(monitor='val_acc', factor=factor, patience=3, verbose=1, min_lr=1e-7)
    
    x = Dense(5, activation='softmax', name='predictions')(model.layers[-2].output)

    M = Model(input=model.input, output=x)

    
    M.compile(loss='categorical_crossentropy',
                optimizer=optimizer, 
                metrics=['acc'])
    history = M.fit_generator(train_generator, 
                                         steps_per_epoch=457//batch_size,
                                         epochs=500,
                                         validation_data=validation_generator,
                                         validation_steps = 197//batch_size,
                                         callbacks=[tensorboard, best_checkpoint, earlystop, reducer]
                                        )
    return history

In [3]:
def loadModel(model):
    if('dense' in model):
        print('densenet')
        return DenseNet201()
    elif('res' in model):
        print('resnet')
        return ResNet50()
    elif('inceptionv3' in model):
        print('inceptionv3')
        return InceptionV3()
    else:
        print('inceptionresnet')
        return InceptionResNetV2()
    

In [4]:
name = 'densenet_0_5'
cnn = loadModel(name)
batch_size = 8

if('inception' in name):
    target_size = (299, 299)
else:
    target_size = (224, 224)
    
folder = '/media/newdrive/20180501/ProcessedImages/'
train_path = folder + 'SampledData/train/'
val_path = folder + 'SampledData/validation/'
test_path = folder + 'test/'

train_df = pd.read_csv(folder + 'sampled_data.csv').set_index('filename')
test_df = pd.read_csv(folder + 'unknown2-targets.csv').set_index('filename')


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=360,
      vertical_flip=True,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        val_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

densenet
Found 457 images belonging to 5 classes.
Found 197 images belonging to 5 classes.


In [5]:
nesterov = SGD(lr=0.001, nesterov=True)
optimizer = SGD(lr=0.001)

    
history = trainCNNModel(cnn, optimizer, 0.5, name)

/home/diegues/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  if __name__ == '__main__':


Epoch 1/500
57/57 [==============================] - 52s 905ms/step - loss: 1.2720 - acc: 0.4870 - val_loss: 1.2773 - val_acc: 0.4792

Epoch 00001: val_acc improved from -inf to 0.47917, saving model to ./weights/densenet_0_5.h5
Epoch 2/500
57/57 [==============================] - 27s 466ms/step - loss: 0.8859 - acc: 0.6712 - val_loss: 1.1147 - val_acc: 0.4896

Epoch 00002: val_acc improved from 0.47917 to 0.48958, saving model to ./weights/densenet_0_5.h5
Epoch 3/500
57/57 [==============================] - 27s 466ms/step - loss: 0.7426 - acc: 0.6976 - val_loss: 0.7345 - val_acc: 0.7812

Epoch 00003: val_acc improved from 0.48958 to 0.78125, saving model to ./weights/densenet_0_5.h5
Epoch 4/500
57/57 [==============================] - 27s 471ms/step - loss: 0.6384 - acc: 0.7478 - val_loss: 1.7613 - val_acc: 0.3229

Epoch 00004: val_acc did not improve from 0.78125
Epoch 5/500
57/57 [==============================] - 26s 462ms/step - loss: 0.6147 - acc: 0.7636 - val_loss: 0.5929 - val_